# <span style="color:#0F19C9">Contents</span>

- [Introduction](#introduction)
- [Importing and loading data](#importing-and-loading-data)
- [Create main dataframe](#create-main-dataframe)
- [Finalizing](#finalizing)

# <span style="color:#0F19C9">Introduction</span>

We have one notebook call `EDA.ipynb`, in it, you can find all the exploration and different plots with context of data. Now is moment to focus on building a machine learning model that helps us to predict the future orders of some users from new columns that we are going to create. The way we fill do it will be creating one dataframe merging all the information to train the model.

One of the goals of the project is going to be better with resource management, mainly memory, because we have to need a lot of it.

# <span style="color:#0F19C9">Importing and loading data</span>

In [50]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [51]:
# Import original data from Instacart
folder = '../Data/Raw/'
files = [file for file in os.listdir(folder)]
dfs = {}

for dataframe in files:
    path = folder + dataframe
    df_name = dataframe.split('.')[0].title()
    dfs[df_name] = pd.read_csv(path)

# Delete temporal variables
del [folder, files, path, df_name, dataframe]

In [52]:
# Merge splitted dataframes
dfs['Orders'] = pd.concat(
    [dfs['Orders_1'], dfs['Orders_2']], ignore_index=True)
to_concat = [dfs[f'Orders_Products__Prior_{i}'] for i in range(1, 18)]
dfs['Orders_Products_Prior'] = pd.concat(to_concat,
                                         ignore_index=True)

# Format column names
for dataframe in dfs.values():
    dataframe.columns = [column.title() for column in dataframe.columns]

# Delete temporals
del [dfs['Orders_1'], dfs['Orders_2']]
for i in range(1, 18):
    del dfs[f'Orders_Products__Prior_{i}']
del [to_concat, dataframe]

# <span style="color:#0F19C9">Create main dataframe</span>

We import all of our raw data, we have not change anything from the original yet. The main idea is to create all the features about an user and the different products that consumes. The features we are going to create are:

- **Times_Bought:** Number of times an user bought a product
- **One_Shot_Ratio_Product:** Products bought just one time divided by the number of users who bought it.
- **Times_In_Last_5:** Number of times the product was bought in last 5 orders.
- **Times_In_Last_5_Ratio:** Number of times the product was bought per order in last 5 orders.
- **Total_Orders:** Number of total orders of the user.
- **First_Ordered:** First number of order of user per product.
- **Ordered_Range:** Range of order numbers that user ordered the product.
- **Order_Ratio:** Times this product was bought in the range of ordered since first bought of this product until last time it was bought.
- **Average_Reordered:** Mean of a product reordered.
- **Avg_To_Cart:** Mean of product order to add to cart.
- **Avg_Order_Size:** Average amount of products per order.

In [53]:
# Create de main dataframe 'df'
df_pivot = dfs['Orders'].merge(dfs['Orders_Products_Prior'],
                               on='Order_Id',
                               how='left')
df = df_pivot.copy()

In [54]:
# Count all the times an user bought a product
df = df.groupby(['User_Id', 'Product_Id'])[['Order_Id']] \
    .count()
df.columns = ['Times_Bought']
df.reset_index(inplace=True)

# Create products bought one time

bought_one_time = df[df['Times_Bought'] == 1] \
    .groupby('Product_Id')[['Times_Bought']] \
    .count()
bought_one_time.columns = ['Bought_One_Time']


# Count the number of users for each one time product

users_that_bought = df.groupby('Product_Id')[['User_Id']] \
    .count()
users_that_bought.columns = ['Users_That_Bought']

# Create new feature column
new_feature = bought_one_time.merge(users_that_bought,
                                    how='left',
                                    on='Product_Id')
new_feature['One_Shot_Ratio_Product'] = new_feature['Bought_One_Time'] / \
    new_feature['Users_That_Bought']
new_feature = new_feature.reset_index()

# Add to main dataframe
df = df.merge(new_feature[['Product_Id', 'One_Shot_Ratio_Product']],
              how='left',
              on='Product_Id')

# Delete temporal variables
del [bought_one_time, users_that_bought]

In [55]:
# Get the 'Order Number Back'
new_pivot = df_pivot.copy()
new_pivot['Order_Number_Back'] = new_pivot \
    .groupby('User_Id')['Order_Number'] \
    .transform('max') - new_pivot['Order_Number'] + 1

# Filter only last 5 orders
new_pivot = new_pivot[new_pivot['Order_Number_Back'] <= 5]
new_pivot = new_pivot.groupby(['User_Id', 'Product_Id'])[['Order_Id']].count()
new_pivot.columns = ['Times_In_Last_5']
new_pivot['Times_In_Last_5_Ratio'] = new_pivot['Times_In_Last_5'] / 5

# Merge new columns
df = df.merge(new_pivot,
              on=['User_Id', 'Product_Id'],
              how='left')

In [56]:
# Create columns
new_pivot = df_pivot.copy()
total_orders = new_pivot \
    .groupby('User_Id')[['Order_Number']].max()
total_orders.columns = ['Total_Orders']

first_order = new_pivot \
    .groupby(['User_Id', 'Product_Id'])[['Order_Number']] \
    .min()
first_order.columns = ['First_Ordered']
first_order.reset_index(inplace=True)

times = new_pivot.groupby(['User_Id', 'Product_Id'])[['Order_Id']].count()
times.columns = ['Times_Bought']

# Merge columns
span = pd.merge(total_orders, first_order,
                on='User_Id',
                how='right')
span['Ordered_Range'] = span['Total_Orders'] - span['First_Ordered'] + 1
new_feature = pd.merge(times, span,
                       on=['User_Id', 'Product_Id'],
                       how='left')
new_feature['Order_Ratio'] = new_feature['Times_Bought'] / \
    new_feature['Ordered_Range']
new_feature.drop('Times_Bought', axis=1, inplace=True)

# Add new features
df = df.merge(new_feature,
              on=['User_Id', 'Product_Id'],
              how='left')

In [57]:
# Create de columns of Average_Reordered and Avg_To_Cart
new_pivot = df_pivot.copy()
new_pivot = new_pivot.groupby('Product_Id')[['Reordered']].mean()
new_pivot.columns = ['Avg_Reordered']
new_pivot['Avg_To_Cart'] = df_pivot.groupby(
    'Product_Id')[['Add_To_Cart_Order']].mean()

# Add new two columns to main dataframe
df = df.merge(new_pivot,
              on='Product_Id',
              how='left')

del [first_order,
     span,
     times,
     total_orders]

In [58]:
# Create feature 'Avg Order Size'
new_pivot = df_pivot.copy()
new_pivot = new_pivot.groupby(['User_Id', 'Order_Id'])[['Product_Id']].count()
new_pivot.columns = ['Size']
new_pivot.reset_index(inplace=True)

new_pivot = new_pivot.groupby('User_Id')[['Size']].mean()
new_pivot.columns = ['Avg_Order_Size']
new_pivot.reset_index(inplace=True)

# Add new feature
df = df.merge(new_pivot,
              on='User_Id',
              how='left')

# <span style="color:#0F19C9">Finalizing</span>

We are going to understand the info of our final dataframe and fill the null values. We ended with a dataframe of just numbers in every column, ideal to train a machine learning model. We have a dataframe of 13'307.953 rows with 13 columns to train the model.

Finally we have to divide the main dataframe to upload to repo without problems.

In [59]:
# Write general info of the main dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   User_Id                 int64  
 1   Product_Id              float64
 2   Times_Bought            int64  
 3   One_Shot_Ratio_Product  float64
 4   Times_In_Last_5         float64
 5   Times_In_Last_5_Ratio   float64
 6   Total_Orders            int64  
 7   First_Ordered           int64  
 8   Ordered_Range           int64  
 9   Order_Ratio             float64
 10  Avg_Reordered           float64
 11  Avg_To_Cart             float64
 12  Avg_Order_Size          float64
dtypes: float64(8), int64(5)
memory usage: 1.3 GB


In [60]:
# Fill values with 0
df.fillna(0, inplace=True)

In [61]:
# Search for duplicated values
df.duplicated().sum()

0

In [62]:
# Show 2 random samples
df.sample(2)

,User_Id,Product_Id,Times_Bought,One_Shot_Ratio_Product,Times_In_Last_5,Times_In_Last_5_Ratio,Total_Orders,First_Ordered,Ordered_Range,Order_Ratio,Avg_Reordered,Avg_To_Cart,Avg_Order_Size
842920,13241,26665.0,2,0.672848,0.0,0.0,51,30,22,0.090909,0.451308,9.572674,9.372549
2871568,44610,48094.0,1,0.698946,0.0,0.0,30,18,13,0.076923,0.437257,9.856348,20.400000


In [63]:
# Divide main dataframe
initial_row = 0
df_size = 700000
final_row = df_size
i = 1

while initial_row <= df.shape[0]:
    df_pivot = df.loc[initial_row: final_row]
    df_pivot.to_csv(f'../Data/Processed/df_{i}.csv', index=False)
    initial_row = final_row
    final_row += df_size
    i += 1